# Load Python libraries

In [1]:
# import whatever libraries I want
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import msm_scripts as ms
import test_system_generator as tsg
import timeit

# Generate data

In [13]:
""" So what I'm doing here is generating reaction coordinates and performing simulations 
for each of coordinates. I am currently generating a bunch of single well potentials 
and 1 double well potential (initial state at transition state)"""

""" I basically run a simulation in the double well (Started at TS) and I 
terminate the run when the system is in one of the stable states. I then
run a simulation of the single well coordinates for an equal number of
time steps."""

simulation_data = [ ] # initialise list to hold my data
num_of_sims_pc = 30 # number of simulations to run per coordinate
num_of_coors = 3 # number of coordinates

numstates = 200 # number of states along each coordinate

T = 50000 # maximum number of timesteps to run simulation for (not really important)

time_step = 0.005 
""" timestep of the simulation (just a matter of scaling how many steps we stay in the 
transition state for) """ 

initstate = int(numstates/2) # my simulations initial state is the middle one

""" Here I'm just defining a list of states which I call 'out'. When the double well
simulation enters one of these states I terminate the run. Currently I have the first
and last 30% of the states. """
term_val1=np.linspace(1,(3/10)*numstates,(3/10)*numstates)
term_val1=[int(x) for x in term_val1]
term_val2=np.linspace(numstates+1-(3/10)*numstates,200,(3/10)*numstates)
term_val2=[int(x) for x in term_val2]
term_val = term_val1 + term_val2

# This a loop to actually generate all my simulation data
for i in range(num_of_sims_pc):
    # define a simulation object in a given potential with a particular biasing force
    sim=tsg.simulation(potential='double_well',sim_length=T,term=term_val,dt=time_step,biasing_protocol=[0,1],num_of_states=numstates,T=300, initial_state = initstate)
    simulation_data.append(sim.generate_limited_data())
    sim_length_tmp = len(simulation_data[-1].data)
    for j in range(num_of_coors-1):
        sim=tsg.simulation(potential='single_well',sim_length=sim_length_tmp,term=term_val,dt=time_step,biasing_protocol=[0,1],num_of_states=numstates,T=300, initial_state = initstate)
        simulation_data.append(sim.generate_data())

/home/adam/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:27: DeprecationWarning: object of type <class 'float'> cannot be safely interpreted as an integer.
/home/adam/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:29: DeprecationWarning: object of type <class 'float'> cannot be safely interpreted as an integer.


(Initialized a data set object containing 1 simulation of length 3622)
(Initialized a data set object containing 1 simulation of length 3622)
(Initialized a data set object containing 1 simulation of length 3622)
(Initialized a data set object containing 1 simulation of length 2073)
(Initialized a data set object containing 1 simulation of length 2073)
(Initialized a data set object containing 1 simulation of length 2073)
(Initialized a data set object containing 1 simulation of length 3242)
(Initialized a data set object containing 1 simulation of length 3242)
(Initialized a data set object containing 1 simulation of length 3242)
(Initialized a data set object containing 1 simulation of length 23275)
(Initialized a data set object containing 1 simulation of length 23275)
(Initialized a data set object containing 1 simulation of length 23275)
(Initialized a data set object containing 1 simulation of length 8090)
(Initialized a data set object containing 1 simulation of length 8090)
(In

# Identifying in vs. out
I'm going to go through the slow coordinates and identify which of the metastable states they went in to. I'll then take the first X frames of each trajectory and try to make a prediction based on that.

In [14]:
""" This is a bit convoluted but basically result_val is a list of 0 and 1 based on
whether the double well trajectory went in or out. """
slow = np.linspace(0,(num_of_coors)*(num_of_sims_pc-1),num_of_sims_pc)
slow = [int(x) for x in slow]

result_val = []
for i in slow:
    #print(simulation_data[i].data[-1])
    if simulation_data[i].data[-1] <= term_val1[-1]:
        result_val.append(0)
    elif simulation_data[i].data[-1] >= term_val2[0]:
        result_val.append(1)
result_val = [int(x) for x in result_val]

In [15]:
""" Next I take the frist one thousand frames of each trajectory and assign it a 0 or 1
depending on whether it eventually went in or out. """

X = []
Y = []

num_frames = 1000

for i in range(num_of_coors):
    tmp = []
    for j in range(num_of_sims_pc):
        tmp = tmp + simulation_data[num_of_coors*j+i].data[:num_frames]
    X.append(tmp)
for j in range(num_of_sims_pc):
    Y = Y + ([result_val[j]]*num_frames)
    
X = np.array(X)
X = X.transpose() 
Y = np.array(Y)

## ML algorithms

Now that I have my data I can start training a model to make predictions.
I need to split in to test and train.

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.4)

### Neural network

Let's try a neural network to do the prediction.

In [17]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(max_iter=200, shuffle=True)

neural_network = mlp.fit(X_train,y_train) 

#Do the predictions for the X_test

y_predicted = neural_network.predict(X_test)

#The results are the difference between the y_test set and the final prediction

results = y_test == y_predicted

In [18]:
np.sum(results)/len(results)

0.7660833333333333

### Logistic regression

Next let's try a logistic regression (basically linear regression but for classification).

In [19]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0, solver='lbfgs', multi_class='ovr').fit(X_train, y_train)
clf.score(X_test,y_test)

0.8361666666666666

In [20]:
""" These coefficients give the relative importance of each coordinate in making
the prediction. """

clf.coef_[0]

array([ 0.45405807, -0.05508962,  0.09918702])

## 2-D potential with significant fast degree of freedom

I want to set up a 2-D potential which has a fast degree of freedom which becomes relevant at the transition state. I want to distinguish that general simulations which explore the landscape won't identify an important fast degree of freedom but that simulations started in the transition state and assigned based on which state it enters first.